In [2]:
from fold import *

In [27]:
# rect/.../rect
a = arange(2, 4, 3)
print(a.shape)
print("---")
print(a)
print("---")
t = a.transpose(1, 2)
print(t.shape)
print("---")
print(t)
print("---")
t = a.transpose(0, 1)
print(t.shape)
print("---")
print(t)
print("---")
t = a.transpose(-3, -1)
print(t.shape)
print("---")
print(t)

(2, 4, 3)
---
[[[ 0,  1,  2],
  [ 3,  4,  5],
  [ 6,  7,  8],
  [ 9, 10, 11]],

 [[12, 13, 14],
  [15, 16, 17],
  [18, 19, 20],
  [21, 22, 23]]]
---
(2, 3, 4)
---
[[[ 0,  3,  6,  9],
  [ 1,  4,  7, 10],
  [ 2,  5,  8, 11]],

 [[12, 15, 18, 21],
  [13, 16, 19, 22],
  [14, 17, 20, 23]]]
---
(4, 2, 3)
---
[[[ 0,  1,  2],
  [12, 13, 14]],

 [[ 3,  4,  5],
  [15, 16, 17]],

 [[ 6,  7,  8],
  [18, 19, 20]],

 [[ 9, 10, 11],
  [21, 22, 23]]]
---
(3, 4, 2)
---
[[[ 0, 12],
  [ 3, 15],
  [ 6, 18],
  [ 9, 21]],

 [[ 1, 13],
  [ 4, 16],
  [ 7, 19],
  [10, 22]],

 [[ 2, 14],
  [ 5, 17],
  [ 8, 20],
  [11, 23]]]


In [26]:
# ragged/rect (sdpa)
a = arange(3, [3, 2, 1], 2, 4)
print(f"B, Seq, num_heads, head_dim = {a.shape}")
print('---')
print(a)
print('---')
t = a.transpose(1, 2)
print(f"B, num_heads, Seq, head_dim = {t.shape}")
print('---')
print(t)

B, Seq, num_heads, head_dim = (3, [3, 2, 1], 2, 4)
---
[[[[ 0,  1,  2,  3],
   [ 4,  5,  6,  7]],

  [[ 8,  9, 10, 11],
   [12, 13, 14, 15]],

  [[16, 17, 18, 19],
   [20, 21, 22, 23]]],


 [[[24, 25, 26, 27],
   [28, 29, 30, 31]],

  [[32, 33, 34, 35],
   [36, 37, 38, 39]]],


 [[[40, 41, 42, 43],
   [44, 45, 46, 47]]]]
---
B, num_heads, Seq, head_dim = (3, 2, Runs(Affine(3, 3, -1), 2), 4)
---
[[[[ 0,  1,  2,  3],
   [ 8,  9, 10, 11],
   [16, 17, 18, 19]],

  [[ 4,  5,  6,  7],
   [12, 13, 14, 15],
   [20, 21, 22, 23]]],


 [[[24, 25, 26, 27],
   [32, 33, 34, 35]],

  [[28, 29, 30, 31],
   [36, 37, 38, 39]]],


 [[[40, 41, 42, 43]],

  [[44, 45, 46, 47]]]]


In [30]:
# ragged/ragged (ok)
a = arange(3, [2, 4, 3], [2, 1, 3, 2, 2, 1, 2, 2, 1])
print(a.shape)
print("---")
print(a)
print("---")
t = a.transpose(1, 2)
print(t.shape)
print("---")
print(t)

(3, [2, 4, 3], [2, 1, 3, 2, 2, 1, 2, 2, 1])
---
[[[ 0,  1],
  [ 2]],

 [[ 3,  4,  5],
  [ 6,  7],
  [ 8,  9],
  [10]],

 [[11, 12],
  [13, 14],
  [15]]]
---
(3, [2, 3, 2], [2, 1, 4, 3, 1, 3, 2])
---
[[[ 0,  2],
  [ 1]],

 [[ 3,  6,  8, 10],
  [ 4,  7,  9],
  [ 5]],

 [[11, 13, 15],
  [12, 14]]]


In [31]:
# ragged/ragged (shear)
a = arange(3, [2, 4, 3], [1, 2, 1, 2, 2, 1, 1, 2, 1])
print(a.shape)
print("---")
print(a)
print("---")
t = a.transpose(1, 2)

(3, [2, 4, 3], [1, 2, 1, 2, 2, 1, 1, 2, 1])
---
[[[ 0],
  [ 1,  2]],

 [[ 3],
  [ 4,  5],
  [ 6,  7],
  [ 8]],

 [[ 9],
  [10, 11],
  [12]]]
---


ValueError: transpose: ragged shape at dim 2 will shear

In [32]:
# ragged/rect
a = arange(3, [2, 4, 3], 2)
print(a.shape)
print("---")
print(a)
print('---')
t = a.transpose(1, 2)
print(t.shape)
print("---")
print(t)

(3, [2, 4, 3], 2)
---
[[[ 0,  1],
  [ 2,  3]],

 [[ 4,  5],
  [ 6,  7],
  [ 8,  9],
  [10, 11]],

 [[12, 13],
  [14, 15],
  [16, 17]]]
---
(3, 2, Runs([2, 4, 3], 2))
---
[[[ 0,  2],
  [ 1,  3]],

 [[ 4,  6,  8, 10],
  [ 5,  7,  9, 11]],

 [[12, 14, 16],
  [13, 15, 17]]]


In [33]:
# rect/ragged
a = arange(3, 2, Runs([2, 4, 3], 2))
print(a.shape)
print("---")
print(a)
print("---")
t = a.transpose(1, 2)
print(t.shape)
print("---")
print(t)

(3, 2, Runs([2, 4, 3], 2))
---
[[[ 0,  1],
  [ 2,  3]],

 [[ 4,  5,  6,  7],
  [ 8,  9, 10, 11]],

 [[12, 13, 14],
  [15, 16, 17]]]
---
(3, [2, 4, 3], 2)
---
[[[ 0,  2],
  [ 1,  3]],

 [[ 4,  8],
  [ 5,  9],
  [ 6, 10],
  [ 7, 11]],

 [[12, 15],
  [13, 16],
  [14, 17]]]


In [36]:
# invalid rect/.../rect
# ragged gap shape incompatible with new outer dim
a = arange(3, [1, 2, 3], 5)
print(a.shape)
print("---")
print(a)
print("---")
t = a.transpose(0, 2)

(3, [1, 2, 3], 5)
---
[[[ 0,  1,  2,  3,  4]],

 [[ 5,  6,  7,  8,  9],
  [10, 11, 12, 13, 14]],

 [[15, 16, 17, 18, 19],
  [20, 21, 22, 23, 24],
  [25, 26, 27, 28, 29]]]
---


ValueError: length_extend: target length 5 not an even multiple of self length 3